In [1]:
import json
from datetime import date
from datetime import datetime,timedelta
import plotly.figure_factory as ff
from copy import deepcopy
import re

In [47]:
#Command to run -> !python scheduling/multiple_dag_devices.py -f dag_info/dag_3_gemm/ -ng 2 -nc 2 -rc 
size = 256
num_queues = 2



In [42]:
with open(".././profiling/dumps_transformer/paramterised_dag_total_heads/2_GPU2_CPU4_0.json") as f:
# with open(".././profiling/dumps_transformer/paramterised_dag_total_heads/2_GPU1_CPU4_0.json") as f:
    timestamps = json.load(f)
kernels = timestamps.keys()
reference_device = {}
reference_host = {}
total_time = 0
global_reference = None

for kernel in kernels:
    device = timestamps[kernel]["device"]
    if device == 'gpu':
        t = timestamps[kernel]["write"]["device_start"]
    else:
        t = timestamps[kernel]["nd_range"]["device_start"]
    if t == -1:
        continue
    if not (device in reference_device):
        reference_device[device] = t
    else:
        reference_device[device] = min(reference_device[device],t)

    t = timestamps[kernel]["write"]["host_queued_end"]
    
    if not (device in reference_host):
        reference_host[device] = t
    else:
        reference_host[device] = min(reference_host[device],t)
        
    t = timestamps[kernel]["write"]["host_queued_start"]
    
    if not global_reference:
        global_reference = t
    else:
        global_reference = min(global_reference,t)

relative_timestamps = deepcopy(timestamps)

# global_reference = None

# for key,value in reference_host.items():
#     if not global_reference:
#         global_reference = value
#     else:
#         global_reference = min(value,global_reference)


for kernel,kernel_timestamps in relative_timestamps.items():
    device = kernel_timestamps["device"]
    for event_type,event_timestamps in kernel_timestamps.items():
        #print(event_type)
        if event_type in ["device","cmdq"]:
            continue
        else:
            #continue
            for sub_event_type in event_timestamps:
                if  sub_event_type[:4] == "host":
                    #event_timestamps[sub_event_type] -= global_reference
                    continue
                else:
                   event_timestamps[sub_event_type] = event_timestamps[sub_event_type] - reference_device[device] + reference_host[device] - global_reference
#                     event_timestamps[sub_event_type] = event_timestamps[sub_event_type] - reference_device[device] + \
#                     kernel_timestamps["write"]["host_queued_start"] - global_reference
                total_time = max(total_time,event_timestamps[sub_event_type])



print "Total Time Taken - ",total_time
print(json.dumps(relative_timestamps,sort_keys=True,indent=1))          


Total Time Taken -  0.025509595871
{
 "FFC_sans_bias0": {
  "cmdq": 0, 
  "device": "gpu", 
  "nd_range": {
   "device_end": 0.018912792205810547, 
   "device_start": 0.018173694610595703
  }, 
  "read": {
   "device_end": -1584508014.8820179, 
   "device_start": -1584508014.8820179
  }, 
  "write": {
   "device_end": 0.018111467361450195, 
   "device_queued": 0.017967939376831055, 
   "device_start": 0.0180661678314209, 
   "host_queued_end": 1584520082.483309, 
   "host_queued_start": 1584520082.468764
  }
 }, 
 "FFC_sans_bias1": {
  "cmdq": 1, 
  "device": "gpu", 
  "nd_range": {
   "device_end": 0.019423484802246094, 
   "device_start": 0.018203020095825195
  }, 
  "read": {
   "device_end": -1584508014.8820179, 
   "device_start": -1584508014.8820179
  }, 
  "write": {
   "device_end": 0.018168926239013672, 
   "device_queued": 0.018018484115600586, 
   "device_start": 0.018124103546142578, 
   "host_queued_end": 1584520082.483309, 
   "host_queued_start": 1584520082.47039
  }
 },

In [43]:
def to_time_delta(timestamp):
    now = datetime.now()
    second = int(timestamp)
    microsecond = (timestamp - float(second))*1e6
    now = now.replace(hour=0, minute=0, second=second, microsecond=int(microsecond))
    return now

In [44]:
def modify_task_labels(df):
    for item in df:
        kernel_id = int(re.findall('\d+',item['Task'])[0])
        if item['Task'].startswith('FFC'):
            item['Task'] = '{}_gemm'.format(kernel_id)
        elif item['Task'].startswith('empty'):
            item['Task'] = '{}_copy'.format(kernel_id)
        elif item['Task'].startswith('softmax'):
            item['Task'] = '{}_softmax'.format(kernel_id)
        elif 'transpose' in item['Task']:
            item['Task'] = '{}_transpose'.format(kernel_id)
        

In [48]:
df = []
kernels = {}




for kernel,events in relative_timestamps.items():
    dev = events["device"]
    Task = "{}-{}".format(kernel,dev)
    ##events -> write,read,nd_range
    ##event  -> device_queued,device_start,device-end
#         for interval in intervals:
#             Resource=event
#             Start = now + timedelta(seconds=interval[0])
#             Finish = now + timedelta(seconds=interval[1])
#             df.append(dict(Task=Task,Start=Start,Finish=Finish,Resource=Resource))

    
    if events["write"]["device_start"] > 0:
        #write_overhead_start = to_time_delta(events["write"]["host_queued"])
        write_event_start = write_overhead_end = to_time_delta(events["write"]["device_start"])
        write_event_end = to_time_delta(events["write"]["device_end"])
        #df.append(dict(Task=Task,Start=write_overhead_start,Finish=write_overhead_end,Resource='overhead'))
        df.append(dict(Task=Task,Start=write_event_start,Finish=write_event_end,Resource='write'))
    

    
    
    exec_event_start  = to_time_delta(events["nd_range"]["device_start"])
    exec_event_end = to_time_delta(events["nd_range"]["device_end"])
    df.append(dict(Task=Task,Start=exec_event_start,Finish=exec_event_end,Resource='nd_range'))
    
    
    if events["read"]["device_start"] > 0:
        read_event_start = to_time_delta(events["read"]["device_start"])
        read_overhead_start = read_event_end = to_time_delta(events["read"]["device_end"])
    #read_overhead_end = to_time_delta(events["read"]["host_end"])
        df.append(dict(Task=Task,Start=read_event_start,Finish=read_event_end,Resource='read'))
    #df.append(dict(Task=Task,Start=read_overhead_start,Finish=read_overhead_end,Resource='overhead'))
    

#print(df)
    
    
# df = [dict(Task="Job-1", Start='2017-01-01', Finish='2017-02-02', Resource='Complete'),
#       dict(Task="Job-1", Start='2017-02-15', Finish='2017-03-15', Resource='Incomplete'),
#       dict(Task="Job-2", Start='2017-01-17', Finish='2017-02-17', Resource='Not Started'),
#       dict(Task="Job-2", Start='2017-01-17', Finish='2017-02-17', Resource='Complete'),
#       dict(Task="Job-3", Start='2017-03-10', Finish='2017-03-20', Resource='Not Started'),
#       dict(Task="Job-3", Start='2017-04-01', Finish='2017-04-20', Resource='Not Started'),
#       dict(Task="Job-3", Start='2017-05-18', Finish='2017-06-18', Resource='Not Started'),
#       dict(Task="Job-4", Start='2017-01-14', Finish='2017-03-14', Resource='Complete')]

colors = {'nd_range': 'rgb(220, 0, 0)',
          'read': (1, 0.9, 0.16),
          'write': 'rgb(0, 255, 100)',
         'overhead': 'rgb(0, 0, 255)'}

df.sort(key = lambda x : int(re.findall('\d+',x['Task'])[0]))
modify_task_labels(df)
fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True,
                      group_tasks=True,width=1000,height=750)

fig.layout['xaxis']['tickformat'] = '%-L'
fig.layout['title'] = 'Transformer 2 heads -  GPU, CQ {0} Total Time {1:.4f}s'.format(num_queues,total_time)
fig.layout['xaxis_title'] = 'Time (milli seconds)'
fig.layout['yaxis_title'] = 'Kernel'
fig.layout['xaxis_range']=[to_time_delta(0.017),to_time_delta(0.030)]
#fig.layout['xaxis_range']=[to_time_delta(0),to_time_delta(0.020)]
#fig.layout['xaxis_range'] = [0,6]
fig.layout['font'] = dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")



fig.show()

In [46]:
df = []
kernels = {}




for kernel,events in relative_timestamps.items():
    dev = events["device"]
    cq = events["cmdq"]
    #print kernel,cq
    Task = "{}_{}".format(dev,cq)
    ##events -> write,read,nd_range
    ##event  -> device_queued,device_start,device-end
#         for interval in intervals:
#             Resource=event
#             Start = now + timedelta(seconds=interval[0])
#             Finish = now + timedelta(seconds=interval[1])
#             df.append(dict(Task=Task,Start=Start,Finish=Finish,Resource=Resource))

    
    if events["write"]["device_start"] > 0:
        #write_overhead_start = to_time_delta(events["write"]["host_queued"])
        write_event_start = write_overhead_end = to_time_delta(events["write"]["device_start"])
        write_event_end = to_time_delta(events["write"]["device_end"])
        #df.append(dict(Task=Task,Start=write_overhead_start,Finish=write_overhead_end,Resource='overhead'))
        df.append(dict(Task=Task,Start=write_event_start,Finish=write_event_end,Resource='write'))
    

    
    if kernel.startswith('FFC'):
        exec_resource = 'gemm'
    elif "transpose" in kernel:
        exec_resource = 'transpose'
    elif 'softmax' in kernel:
        exec_resource = 'softmax'
    else:
        exec_resource = 'copy'
    
    exec_event_start  = to_time_delta(events["nd_range"]["device_start"])
    exec_event_end = to_time_delta(events["nd_range"]["device_end"])
    df.append(dict(Task=Task,Start=exec_event_start,Finish=exec_event_end,Resource=exec_resource))
    
    
    if events["read"]["device_start"] > 0:
        read_event_start = to_time_delta(events["read"]["device_start"])
        read_overhead_start = read_event_end = to_time_delta(events["read"]["device_end"])
    #read_overhead_end = to_time_delta(events["read"]["host_end"])
        df.append(dict(Task=Task,Start=read_event_start,Finish=read_event_end,Resource='read'))
    #df.append(dict(Task=Task,Start=read_overhead_start,Finish=read_overhead_end,Resource='overhead'))
    

#print(df)
    
    
# df = [dict(Task="Job-1", Start='2017-01-01', Finish='2017-02-02', Resource='Complete'),
#       dict(Task="Job-1", Start='2017-02-15', Finish='2017-03-15', Resource='Incomplete'),
#       dict(Task="Job-2", Start='2017-01-17', Finish='2017-02-17', Resource='Not Started'),
#       dict(Task="Job-2", Start='2017-01-17', Finish='2017-02-17', Resource='Complete'),
#       dict(Task="Job-3", Start='2017-03-10', Finish='2017-03-20', Resource='Not Started'),
#       dict(Task="Job-3", Start='2017-04-01', Finish='2017-04-20', Resource='Not Started'),
#       dict(Task="Job-3", Start='2017-05-18', Finish='2017-06-18', Resource='Not Started'),
#       dict(Task="Job-4", Start='2017-01-14', Finish='2017-03-14', Resource='Complete')]

colors = {'copy': '#ff00ff',
          'read': (1, 0.9, 0.16),
          'write': 'rgb(0, 255, 100)',
         'overhead': 'rgb(0, 0, 255)',
         'gemm': '#dd3069',
         'transpose': '#a9a9d9',
         'softmax': '#3399cc'}

df.sort(key = lambda x : x['Task'])
#modify_task_labels(df)
fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True,
                      group_tasks=True,width=1000,height=750)

fig.layout['xaxis']['tickformat'] = '%L'
fig.layout['title'] = 'Transformer 2 heads -  GPU-GPU, CQ {0} Total Time {1:.4f}s'.format(num_queues,total_time)
fig.layout['xaxis_title'] = 'Time (milli seconds)'
fig.layout['yaxis_title'] = 'Command Queue'
#fig.layout['xaxis_range']=[to_time_delta(0.017),to_time_delta(0.025)]
#fig.layout['xaxis_range'] = [0,6]
fig.layout['font'] = dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")



fig.show()